In [1]:
import pandas as pd
import numpy as np
import itertools
import sys
from ast import literal_eval

In [2]:
prod_num_map = pd.read_csv('../../../data/cabot_data/sprint_2/prod_num_map.csv',
                           header=None, index_col=4)

In [3]:
prod_num_map['prod_num'] = prod_num_map.index
prod_num_map.index = prod_num_map['prod_num']
prod_num_map = prod_num_map.drop(['prod_num'], axis=1)

In [4]:
prod_cols = prod_num_map.index.tolist()
num_products = max(prod_num_map.index.tolist())+1
prod_cols.append(num_products)

In [5]:
# helper vars to store ranges
units = prod_num_map[0].unique().tolist()
weeks = prod_num_map[1].unique().tolist()
dows = prod_num_map[2].unique().tolist()
lens = prod_num_map[3].unique().tolist()

In [6]:
# week orderings
week_orderings = []
for i in range(0, len(weeks)-1):
    week_orderings.append((weeks[i], weeks[i+1])) # normal week ordering
    week_orderings.append((weeks[i+1], weeks[i])) # reverse week ordering

In [7]:
# unit orderings
unit_part_orderings = [('CD', 'CK', 'DD', 'DK', 'DKB'),
                       ('CD', 'DD'),
                       ('CD', 'DK', 'DKB')]
unit_orderings = []
for o in unit_part_orderings:
    unit_orderings.append(o)
    unit_orderings.append(o[::-1])
unit_orderings.append(['2BV'])
unit_orderings.append(['4BV'])

In [8]:
# combined orderings using cartesian products
comb_ordering = []
for cart_prod_1 in itertools.product(week_orderings, unit_orderings):
        
    # week priority
    temp_order = []
    for cart_prod_2 in itertools.product(cart_prod_1[0], cart_prod_1[1]):
        temp_order.append(cart_prod_2)
    comb_ordering.append(temp_order)
    
    # include unit priority if there's more than 1 unit selected
    if (len(cart_prod_1[1]) > 1): 
        temp_order = []
        for cart_prod_2 in itertools.product(cart_prod_1[1], cart_prod_1[0]):
            temp_order.append(cart_prod_2[::-1])
        comb_ordering.append(temp_order)

In [9]:
# blank cust types df
num_rows = len(comb_ordering)*len(dows)*len(lens)
cust_types = pd.DataFrame(index=range(1, num_rows+1),
                          columns=prod_cols)

# blank row
blank_type = pd.Series([0 for x in range(1, num_products+1)],
                       index = prod_cols)

In [10]:
# helper load bar function
def progress(count, total, status=''):
    bar_len = 60
    filled_len = int(round(bar_len * count / float(total)))

    percents = round(100.0 * count / float(total), 1)
    bar = '=' * filled_len + '-' * (bar_len - filled_len)

    sys.stdout.write('[%s] %s%s ...%s\r' % (bar, percents, '%', status))
    sys.stdout.flush()
total = num_rows

In [11]:
# populate cust types
idx_counter = 1
for stay_length in lens:
    for dow in dows:
        for order in comb_ordering:
            progress(idx_counter, total, status='Filling in types')
            # making a series first makes it go faster
            new_row = blank_type.copy()
            for col_index, value in enumerate(order):
                new_row.loc[col_index+1] = (value[1], value[0], dow, stay_length)
            cust_types.loc[idx_counter] = new_row
            idx_counter += 1

In [12]:
cust_types.to_csv('../../../data/cabot_data/sprint_2/types_s2_raw.csv')

In [13]:
cust_types_m = pd.read_csv('../../../data/cabot_data/sprint_2/types_s2_raw.csv', index_col=[0])

In [14]:
# import col tuple to number map
prod_num_map = pd.read_csv('../../../data/cabot_data/sprint_2/prod_num_map.csv',
                           header=None,
                           index_col=[0,1,2,3])

prod_num_map = pd.Series(prod_num_map.iloc[:,0])
prod_num_map = prod_num_map.to_dict()

In [15]:
# convert strings to tuple and map to product number
i = 1
while len(cust_types_m[str(i)].nonzero()[0]) > 0:
    cust_types_m[str(i)] = [literal_eval(x) for x in cust_types_m[str(i)]]
    cust_types_m[str(i)] = cust_types_m[str(i)].map(prod_num_map)
    i = i + 1

In [16]:
# misc postprocessing
cust_types_m = cust_types_m.fillna(0).astype(int)
cust_types_m.index.names = ['cust_types']

In [17]:
cust_types_m.to_csv('../../../data/cabot_data/sprint_2/types_s2.csv')

In [19]:
cust_types_m

,1,2,3,4,5,6,7,8,9,10,...,4892,4893,4894,4895,4896,4897,4898,4899,4900,4901
cust_types,,,,,,,,,,,,,,,,,,,,,
1,1401,2101,2801,3501,4201,1429,2129,2829,3529,4229,...,0,0,0,0,0,0,0,0,0,0
2,1401,1429,2101,2129,2801,2829,3501,3529,4201,4229,...,0,0,0,0,0,0,0,0,0,0
3,4201,3501,2801,2101,1401,4229,3529,2829,2129,1429,...,0,0,0,0,0,0,0,0,0,0
4,4201,4229,3501,3529,2801,2829,2101,2129,1401,1429,...,0,0,0,0,0,0,0,0,0,0
5,1401,2801,1429,2829,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,1401,1429,2801,2829,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,2801,1401,2829,1429,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,2801,2829,1401,1429,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,1401,3501,4201,1429,3529,4229,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
